In [15]:
from dotenv import load_dotenv
import boto3
import os
import polars as pl

In [16]:
load_dotenv()

True

In [17]:
s3 = boto3.client('s3',
    endpoint_url='https://'+os.getenv('DUCKDB_S3_ENDPOINT'),
)

In [18]:
# List all objects in the bucket
objects = s3.list_objects_v2(Bucket='archiva-apagones')
# objects

In [19]:


def get_all_objects():
    
    # Initial request to list objects
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket='archiva-apagones')

    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                yield obj


In [20]:
for obj in objects['Contents']:
    print(obj)
    break

{'Key': 'genera/data_source/2023-12-26/genera_data_source__2023-12-26T21-48-54-0400.js', 'LastModified': datetime.datetime(2023, 12, 27, 1, 48, 55, 975000, tzinfo=tzutc()), 'ETag': '"cc75570b565784e1e73e04eeef2a323c"', 'Size': 10311, 'StorageClass': 'STANDARD'}


In [21]:
objects_df = (
    pl.DataFrame(get_all_objects())
    .with_columns([
        pl.when(pl.col('Key').str.starts_with('genera'))
        .then(pl.col('Key').str.split('/').list.slice(0,2).list.join('/'))
        .otherwise(pl.col('Key').str.split('/').list.get(0))
        .alias('dataset')
    ])
)
objects_df

Key,LastModified,ETag,Size,StorageClass,dataset
str,datetime[μs],str,i64,str,str
"""genera/data_so…",2023-12-27 01:48:55.975,"""""cc75570b56578…",10311,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 02:35:43.279,"""""29739f74bbdb6…",10305,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 02:40:17.820,"""""fb81ee8ba448e…",10304,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 02:45:15.713,"""""2ffe81b9af089…",10300,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 02:50:16.624,"""""4609efd807273…",10298,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 02:55:13.952,"""""fefcf9dfd4089…",10297,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 03:00:17.059,"""""9dc57786f9846…",10299,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 03:05:19.057,"""""19e51d1cbc1b3…",10298,"""STANDARD""","""genera/data_so…"
"""genera/data_so…",2023-12-27 03:10:17.064,"""""f34eedd77e21d…",10298,"""STANDARD""","""genera/data_so…"


In [26]:
(
    objects_df
    .group_by('dataset')
    .agg([
        pl.count(),
        pl.sum('Size')
    ])
    .with_columns([
        (pl.col('Size')/1e6).alias('Size (MB)'),
        (pl.col('Size')/pl.col('count')).round().alias('Avg. Size'),
        (pl.col('Size')/pl.col('count') * 288 / 1e6).round(3).alias('Avg. Daily Size (MB)'),
    ])
    .sort('Size (MB)', descending=True)
).to_pandas()

,dataset,count,Size,Size (MB),Avg. Size,Avg. Daily Size (MB)
0,genera/data_source,1039,10657224,10.657224,10257.0,2.954
1,regions_without_service,3172,6181911,6.181911,1949.0,0.561
2,outage_towns,212,634805,0.634805,2994.0,0.862
3,penguins.csv,1,13478,0.013478,13478.0,3.882
4,regions_without_service.json,1,1947,0.001947,1947.0,0.561
